In [1]:
from sentiment_modules import compute_metrics, dataset, sentiment
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import EarlyStoppingCallback
from transformers import Trainer, TrainingArguments

2023-12-20 18:11:50.655383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sen = sentiment("restaurant-reviews.csv", "title", "text", "rating")

In [3]:
checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BertForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels = 5,
    id2label = {
        0: "1 star",
        1: "2 stars",
        2: "3 stars",
        3: "4 stars",
        4: "5 stars"
    },
    label2id = {
        "1 star": 0,
        "2 stars": 1,
        "3 stars": 2,
        "4 stars": 3,
        "5 stars": 4
    }
)

In [4]:
X = [sen.bert_prep(text_type) for text_type in [sen.title, sen.text]]
X = [f"{title}. {text}" for title, text in zip(X[0], X[1])]
y = list(sen.score - 1)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                  y, 
                                                  test_size = 0.2,
                                                  random_state = 42,
                                                  shuffle = True,
                                                  stratify = y)
X_train_tokenized = tokenizer(X_train, padding = True, truncation = True, max_length = 512)
X_val_tokenized = tokenizer(X_val, padding = True, truncation = True, max_length = 512)

In [6]:
train_dataset = dataset(X_train_tokenized, y_train)
val_dataset = dataset(X_val_tokenized, y_val)

In [7]:
model_name = "bert-base-multilingual-uncased-finetuned-sentiment"
args = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 8, 
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    logging_strategy = "epoch",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3, 
    learning_rate = 2e-5,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    load_best_model_at_end = True,
    metric_for_best_model = "balanced_accuracy"
)

In [8]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [9]:
trainer.train()

  0%|          | 0/3200 [00:00<?, ?it/s]

{'loss': 0.9462, 'learning_rate': 1.9444444444444445e-05, 'epoch': 1.0}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 1.0949993133544922, 'eval_balanced_accuracy': 0.4680758957354702, 'eval_runtime': 331.6132, 'eval_samples_per_second': 0.603, 'eval_steps_per_second': 0.302, 'epoch': 1.0}
{'loss': 0.9294, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 2.0643770694732666, 'eval_balanced_accuracy': 0.632619508151423, 'eval_runtime': 324.1539, 'eval_samples_per_second': 0.617, 'eval_steps_per_second': 0.308, 'epoch': 6.0}
{'loss': 0.0182, 'learning_rate': 2.7777777777777783e-06, 'epoch': 7.0}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 2.15651273727417, 'eval_balanced_accuracy': 0.6078060237634706, 'eval_runtime': 325.4384, 'eval_samples_per_second': 0.615, 'eval_steps_per_second': 0.307, 'epoch': 7.0}
{'train_runtime': 35864.9006, 'train_samples_per_second': 0.178, 'train_steps_per_second': 0.089, 'train_loss': 0.4305599107061114, 'epoch': 7.0}


TrainOutput(global_step=2800, training_loss=0.4305599107061114, metrics={'train_runtime': 35864.9006, 'train_samples_per_second': 0.178, 'train_steps_per_second': 0.089, 'train_loss': 0.4305599107061114, 'epoch': 7.0})

In [10]:
trainer.evaluate()

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 1.6647740602493286,
 'eval_balanced_accuracy': 0.6791563046882196,
 'eval_runtime': 316.8158,
 'eval_samples_per_second': 0.631,
 'eval_steps_per_second': 0.316,
 'epoch': 7.0}

In [11]:
trainer.save_model(model_name)